# 😷 Real-Time Face Mask Detection with Colab Webcam

This notebook will download a pretrained mask detection model, capture an image from your webcam, detect faces, and classify them as Mask/No Mask. Ready to run out of the box!

In [9]:
# ===================================
# Fake News Detection System in Colab
# ===================================

# 1) Install required NLTK resources
import nltk
nltk.download("stopwords", force=True)

# 2) Imports
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 3) Load datasets
true_df = pd.read_csv("/content/FakeNews_True.csv")
fake_df = pd.read_csv("/content/FakeNews_Fake.csv")

# 4) Add labels: 1=real, 0=fake
true_df["label"] = 1
fake_df["label"] = 0

# 5) Combine datasets & shuffle
df = pd.concat([true_df, fake_df], ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# 6) Preprocessing function WITHOUT nltk.word_tokenize()
def clean_text(text):
    # Lowercase & split
    tokens = text.lower().split()
    # Remove punctuation and keep only alphabetic words
    tokens = [word.strip(string.punctuation) for word in tokens if word.isalpha() or word.isalnum()]
    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words("english")]
    return " ".join(tokens)

# 7) Apply preprocessing
df["clean_text"] = df["text"].apply(clean_text)

# 8) Split data
X_train, X_test, y_train, y_test = train_test_split(
    df["clean_text"], df["label"], test_size=0.2, random_state=42
)

# 9) Vectorize with TF-IDF
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# 10) Train Logistic Regression model
model = LogisticRegression()
model.fit(X_train_vec, y_train)

# 11) Evaluate on test set
y_pred = model.predict(X_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# 12) Prediction function
def predict_news(news_text):
    clean = clean_text(news_text)
    vec = vectorizer.transform([clean])
    pred = model.predict(vec)[0]
    return "Real" if pred == 1 else "Fake"

# 13) Example prediction
test_news = "The president announced a new health policy today."
print("\nExample prediction:")
print(f"Input: {test_news}")
print(f"Prediction: {predict_news(test_news)}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Accuracy: 0.5

Confusion Matrix:
 [[3 0]
 [4 1]]

Classification Report:
               precision    recall  f1-score   support

           0       0.43      1.00      0.60         3
           1       1.00      0.20      0.33         5

    accuracy                           0.50         8
   macro avg       0.71      0.60      0.47         8
weighted avg       0.79      0.50      0.43         8


Example prediction:
Input: The president announced a new health policy today.
Prediction: Fake
